# Projecto ETL Migracion a PostgreSQL

In [78]:
# Importacion de librerias
import pandas as pd
import numpy as np
import os

In [79]:
# Definicion de paths
path_ = os.getcwd()
dataset_name = 'jio_mart_items.csv'

In [80]:
# Carga de datos rawdata products
products = pd.read_csv(os.path.join(path_, dataset_name))

In [81]:
# Create table category
list_category = products['category'].unique()
size_category = len(list_category)
list_id_category = list(range(1,size_category+1))
category = pd.DataFrame({'category_name': list_category,
                        'category_id': list_id_category})

In [82]:
# Create table sub-category
list_sub_category = products['sub_category'].unique()
size_sub_category = len(list_sub_category)
list_id_sub_category = list(range(1,size_sub_category+1))
sub_category = pd.DataFrame({'sub_category_name': list_sub_category,
                            'sub_category_id': list_id_sub_category})

In [83]:
# Trabajar en el update para manejar los productos que se repiten
products_value_counts = products['items'].value_counts().reset_index()
products_value_counts = products_value_counts.rename(columns={'items': 'count', 'index': 'items'})
products = pd.merge(products, products_value_counts, on='items', how='left')
products = products.query('count == 1.0')

    

In [84]:
products

,category,sub_category,href,items,price,count
1,Groceries,Fruits & Vegetables,https://www.jiomart.com/c/groceries/fruits-veg...,Tender Coconut Cling Wrapped (1 pc) (Approx 90...,49.0,1.0
2,Groceries,Fruits & Vegetables,https://www.jiomart.com/c/groceries/fruits-veg...,Mosambi 1 kg,69.0,1.0
3,Groceries,Fruits & Vegetables,https://www.jiomart.com/c/groceries/fruits-veg...,Orange Imported 1 kg,125.0,1.0
4,Groceries,Fruits & Vegetables,https://www.jiomart.com/c/groceries/fruits-veg...,Banana Robusta 6 pcs (Box) (Approx 800 g - 110...,44.0,1.0
6,Groceries,Fruits & Vegetables,https://www.jiomart.com/c/groceries/fruits-veg...,Pineapple Queen 1 pc (Approx 700 g - 1200 g),84.0,1.0
...,...,...,...,...,...,...
162308,Jewellery,Fine Jewellery,https://www.jiomart.com/c/jewellery/fine-jewel...,Reliance Jewels Ag 99.9 5.6 gm Ganesha Silver ...,1669.0,1.0
162309,Jewellery,Fine Jewellery,https://www.jiomart.com/c/jewellery/fine-jewel...,Reliance Jewels Ag 99.9 4.58 gm Ganesha Silver...,1391.0,1.0
162310,Jewellery,Fine Jewellery,https://www.jiomart.com/c/jewellery/fine-jewel...,Reliance Jewels Ag 99.9 2.31 gm Ganesha Silver...,834.0,1.0
162311,Jewellery,Fine Jewellery,https://www.jiomart.com/c/jewellery/fine-jewel...,Reliance Jewels Ag 99.9 7.76 gm Ganesha Silver...,2276.0,1.0
